In [1]:
# Import required libraries.

import pandas as pd

In [2]:
# Read all datasets.

jan  = pd.read_csv("datasets/01_flight_details_january.csv")
feb  = pd.read_csv("datasets/02_flight_details_february.csv")
mar  = pd.read_csv("datasets/03_flight_details_march.csv")
apr  = pd.read_csv("datasets/04_flight_details_april.csv")
may  = pd.read_csv("datasets/05_flight_details_may.csv")
jun  = pd.read_csv("datasets/06_flight_details_june.csv")
jul  = pd.read_csv("datasets/07_flight_details_july.csv")
aug  = pd.read_csv("datasets/08_flight_details_august.csv")
sep  = pd.read_csv("datasets/09_flight_details_september.csv")
octo = pd.read_csv("datasets/10_flight_details_october.csv")
nov  = pd.read_csv("datasets/11_flight_details_november.csv")
dec  = pd.read_csv("datasets/12_flight_details_december.csv")


In [183]:
# Make list of dasets to make iterating easy.

df_list = [jan, feb, mar, apr, may, jun, jul, aug, sep, octo, nov, dec]

In [184]:
feb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 410517 entries, 0 to 410516
Data columns (total 25 columns):
FL_DATE                410517 non-null object
AIRLINE_ID             410517 non-null int64
TAIL_NUM               409907 non-null object
FL_NUM                 410517 non-null int64
ORIGIN                 410517 non-null object
ORIGIN_CITY_NAME       410517 non-null object
DEST                   410517 non-null object
DEST_CITY_NAME         410517 non-null object
CRS_DEP_TIME           410517 non-null int64
DEP_TIME               404337 non-null float64
DEP_DELAY              404335 non-null float64
DEP_DELAY_GROUP        404335 non-null float64
CRS_ARR_TIME           410517 non-null int64
ARR_TIME               403966 non-null float64
ARR_DELAY              403225 non-null float64
ARR_DELAY_GROUP        403225 non-null float64
CANCELLED              410517 non-null float64
DIVERTED               410517 non-null float64
AIR_TIME               403225 non-null float64
CARRIER_DE

In [185]:
# Make copies of datasets.
df_copy_list = []
for df in df_list:
    df_copy_list.append(df.copy())

# Deconstruct copy list for redable access to elements.
jan_copy, feb_copy, mar_copy, apr_copy, may_copy, jun_copy, jul_copy, aug_copy, sep_copy, octo_copy, nov_copy, dec_copy = df_copy_list

In [186]:
# Convert columns to appropriate datatypes.

for df in df_copy_list:
    
    df.dropna(subset=["DEP_TIME", "ARR_TIME"], inplace=True)
    
    df.FL_DATE = pd.to_datetime(df.FL_DATE)
    
    df.AIRLINE_ID = df.AIRLINE_ID.astype("str")
    
    df.FL_NUM = df.FL_NUM.astype("str")
    
    df.CRS_DEP_TIME = df.CRS_DEP_TIME.astype("str").str.zfill(4).replace("2400", "0000")
    df.CRS_ARR_TIME = df.CRS_ARR_TIME.astype("str").str.zfill(4).replace("2400", "0000")    
    df.CRS_DEP_TIME = pd.to_datetime(df.CRS_DEP_TIME, format="%H%M").dt.time
    df.CRS_ARR_TIME = pd.to_datetime(df.CRS_ARR_TIME, format="%H%M").dt.time
    
    df.DEP_TIME = df.DEP_TIME.astype("int").astype("str").str.zfill(4).replace("2400", "0000")
    df.ARR_TIME = df.ARR_TIME.astype("int").astype("str").str.zfill(4).replace("2400", "0000")    
    df.DEP_TIME = pd.to_datetime(df.DEP_TIME, format="%H%M").dt.time
    df.ARR_TIME = pd.to_datetime(df.ARR_TIME, format="%H%M").dt.time
    
    df.DEP_DELAY_GROUP = df.DEP_DELAY_GROUP.astype("int", errors="ignore")
    df.ARR_DELAY_GROUP = df.ARR_DELAY_GROUP.astype("int", errors="ignore")
    
    df.DEP_DELAY = df.DEP_DELAY.astype("int", errors="ignore")
    df.ARR_DELAY = df.ARR_DELAY.astype("int", errors="ignore")
    
    df.CANCELLED = df.CANCELLED.astype("bool")
    df.DIVERTED = df.DIVERTED.astype("bool")
    
    df["FLIGHT_ID"] = df.index.astype("str").str.zfill(6) + "-" + df["AIRLINE_ID"] + df["TAIL_NUM"]
    
    df.drop(["AIRLINE_ID", "TAIL_NUM", "Unnamed: 24"], axis=1, inplace=True)
    

In [187]:
feb_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 403966 entries, 0 to 410516
Data columns (total 23 columns):
FL_DATE                403966 non-null datetime64[ns]
FL_NUM                 403966 non-null object
ORIGIN                 403966 non-null object
ORIGIN_CITY_NAME       403966 non-null object
DEST                   403966 non-null object
DEST_CITY_NAME         403966 non-null object
CRS_DEP_TIME           403966 non-null object
DEP_TIME               403966 non-null object
DEP_DELAY              403966 non-null int32
DEP_DELAY_GROUP        403966 non-null int32
CRS_ARR_TIME           403966 non-null object
ARR_TIME               403966 non-null object
ARR_DELAY              403225 non-null float64
ARR_DELAY_GROUP        403225 non-null float64
CANCELLED              403966 non-null bool
DIVERTED               403966 non-null bool
AIR_TIME               403225 non-null float64
CARRIER_DELAY          64264 non-null float64
WEATHER_DELAY          64264 non-null float64
NAS_DELAY 

In [188]:
feb_copy.head()

,FL_DATE,FL_NUM,ORIGIN,ORIGIN_CITY_NAME,DEST,DEST_CITY_NAME,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DELAY_GROUP,...,ARR_DELAY_GROUP,CANCELLED,DIVERTED,AIR_TIME,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,FLIGHT_ID
0,2017-02-01,33,BTV,"Burlington, VT",JFK,"New York, NY",19:07:00,18:57:00,-10,-1,...,-1.0,False,False,66.0,NaN,NaN,NaN,NaN,NaN,000000-20409N267JB
1,2017-02-01,34,JFK,"New York, NY",BTV,"Burlington, VT",17:06:00,16:56:00,-10,-1,...,-1.0,False,False,49.0,NaN,NaN,NaN,NaN,NaN,000001-20409N267JB
2,2017-02-01,42,LGB,"Long Beach, CA",RNO,"Reno, NV",09:12:00,09:09:00,-3,-1,...,-1.0,False,False,63.0,NaN,NaN,NaN,NaN,NaN,000002-20409N632JB
3,2017-02-01,43,RNO,"Reno, NV",LGB,"Long Beach, CA",11:20:00,11:19:00,-1,-1,...,-1.0,False,False,64.0,NaN,NaN,NaN,NaN,NaN,000003-20409N632JB
4,2017-02-01,49,BOS,"Boston, MA",SAV,"Savannah, GA",10:40:00,10:33:00,-7,-1,...,-1.0,False,False,141.0,NaN,NaN,NaN,NaN,NaN,000004-20409N351JB


In [189]:
for df in df_copy_list:
    df["ORIGIN_CITY"], df["ORIGIN_STATE"] = df.ORIGIN_CITY_NAME.str.split(", ").str
    df["DEST_CITY"], df["DEST_STATE"] = df.DEST_CITY_NAME.str.split(", ").str
    df.drop(["ORIGIN_CITY_NAME", "DEST_CITY_NAME"], inplace=True, axis=1)

In [190]:
apr_copy.head()

,FL_DATE,FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DELAY_GROUP,CRS_ARR_TIME,ARR_TIME,...,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,FLIGHT_ID,ORIGIN_CITY,ORIGIN_STATE,DEST_CITY,DEST_STATE
0,2017-04-01,411,PHX,ORD,12:18:00,12:58:00,40,2,17:49:00,18:44:00,...,0.0,0.0,15.0,0.0,40.0,000000-19805N926AA,Phoenix,AZ,Chicago,IL
1,2017-04-01,411,SNA,PHX,10:02:00,09:57:00,-5,-1,11:18:00,11:09:00,...,NaN,NaN,NaN,NaN,NaN,000001-19805N915AA,Santa Ana,CA,Phoenix,AZ
2,2017-04-01,412,OGG,PHX,21:15:00,21:00:00,-15,-1,05:49:00,05:33:00,...,NaN,NaN,NaN,NaN,NaN,000002-19805N204AA,Kahului,HI,Phoenix,AZ
3,2017-04-01,413,DTW,PHX,11:00:00,10:53:00,-7,-1,12:29:00,12:09:00,...,NaN,NaN,NaN,NaN,NaN,000003-19805N658AA,Detroit,MI,Phoenix,AZ
4,2017-04-01,413,PHX,SMF,13:35:00,13:28:00,-7,-1,15:38:00,15:26:00,...,NaN,NaN,NaN,NaN,NaN,000004-19805N658AA,Phoenix,AZ,Sacramento,CA
